In [1]:
import requests
import arcpy
import json
import zipfile
from os import listdir, mkdir
from os.path import exists

# Adding coords

Setting up two points with 8km buffer to later clip the datasets.

In [2]:
points = [
    [-92.148796, 44.127985],
    [-92.043726, 44.052865],
]


output_fc = 'C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/Lab2_2.gdb/PointFeature'

arcpy.CreateFeatureclass_management(
    out_path='C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/Lab2_2.gdb',
    out_name='PointFeature',
    geometry_type='POINT',
    spatial_reference=arcpy.SpatialReference(4326)
)

with arcpy.da.InsertCursor(output_fc, ['SHAPE@']) as cursor:
    for longitude, latitude in points:
        point = arcpy.Point(longitude, latitude)
        point_geometry = arcpy.PointGeometry(point, arcpy.SpatialReference(4326))
        cursor.insertRow([point_geometry])
    
buffer_fcs = []
for i, point in enumerate(points):
    buffer_fc = 'C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/Lab2_2.gdb/BufferFeature'.format(i + 1)
    buffer_fcs.append(buffer_fc)
    arcpy.Buffer_analysis(output_fc, buffer_fc, '8000 Meters')

# DEM
1. Calculating the slope;
2. Clipping the layer;
2. Reclassifying the data (the greater the slop the greater the weight).

In [3]:
raster_layer = 'C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/slope'  

clipped_raster = 'C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/Lab2_2.gdb/ClippedRaster'

arcpy.management.Clip(raster_layer, "#", clipped_raster, buffer_fc, "#", "ClippingGeometry", "NO_MAINTAIN_EXTENT")

slope_reclass = arcpy.sa.Reclassify("ClippedRaster", "Value", "0 7 1;7 14 2;14 21 3;21 28 4;28 35 5;35 42 6;42 49 7;49 56 8;56 63 9; 63 76 10", "NODATA"); 

# NLCD
1. Clipping the dataset;
2. Reclassifying the data (the higher the weight the worse).

In [4]:
'11 10' # Open Water
'21 0' # Developed, Open Space
'22 0' #Developed, Low Intensity
'23 0' #Developed, Medium Intensity
'24 0' #Developed, High Intensity
'31 0' #Barren Land
'41 2' #Deciduous Forest
'42 2' #Evergreen Forest
'43 2' #Mixed Forest
'52 5' #Shrub/Scrub
'71 8' #Herbaceous
'81 8' #Hay/Pasture
'82 10' #Cultivated Crops
'90 2' #Woody Wetlands
'95 2' #Emergent Herbaceous Wetlands

'95 2'

In [5]:
path = r'C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_2\lc\NLCD 2019 - Land Cover.lyr' 

out_path = 'C:/Users/rozan012/Documents/ArcGIS/Projects/Lab2_2/Lab2_2.gdb/ClippedNLCD'

arcpy.management.Clip(path, "#", out_path, buffer_fcs[0], "#", "ClippingGeometry", "NO_MAINTAIN_EXTENT")

lc_reclass = arcpy.sa.Reclassify("ClippedNLCD", "Value", "11 10; 21 0; 22 0; 23 0; 24 0; 31 0; 41 2; 42 2; 43 2; 52 5; 71 8; 81 8; 82 10; 90 2; 95 2", "DATA")

# Generating 3 cost surfaces

In [8]:
for idx, w in enumerate([0.25, 0.5, 0.75]):
    lc_weight = w
    slope_weight = 1-w
    cost_surf = arcpy.ia.RasterCalculator([lc_reclass, slope_reclass], 
                                          ['lc_reclass', 'slope_reclass'],
                                          expression=f"({lc_weight} * lc_reclass) + ({slope_weight} * slope_reclass)")
    output_path = f"C:/Users/rozan012/Documents/ArcGIS/Projects/Lab3_1/cost_surface_{idx}.tif"
    cost_surf.save(output_path)

# Creating routes

In [11]:
for idx, w in enumerate([0.25, 0.5, 0.75]):
    lc_weight = w
    slope_weight = 1-w
    cost_surf = arcpy.ia.RasterCalculator([lc_reclass, slope_reclass], 
                                          ['lc_reclass', 'slope_reclass'],
                                          expression=f"({lc_weight} * lc_reclass) + ({slope_weight} * slope_reclass)")
    
    opt_path = arcpy.sa.OptimalRegionConnections("PointFeature", f'cost_surf_{idx}', in_cost_raster = cost_surf)